In [1]:
import torch
import pandas as pd
import numpy as np
import transformers
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

from transformers import BertTokenizer, AutoModel

from Preprocessing import preprocess_text

c:\Users\LENOVO\GitHub\Jurnal-Clustering\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('data/data_sinta_cleaned.csv')
df

,jid,desc,title,date,journal,data,data_cleaned,word,jid_umum
0,1,Belakangan ini Lembaga PAUD berbondong-bondong...,Kemampuan Bahasa Inggris Awal pada Periode Lin...,2023-10-05,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,Kemampuan Bahasa Inggris Awal pada Periode Lin...,kemampuan bahasa inggris awal pada periode lin...,17,1
1,1,The assessment process carried out in schools ...,Strengthening Early Childhood Learning Outcome...,2022-06-09,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,Strengthening Early Childhood Learning Outcome...,strengthening early childhood learning outcome...,17,1
2,1,Kewajiban guru PAUD untuk memberikan layanan b...,Eksplorasi Deskriptif tentang Layanan Bimbinga...,2023-06-07,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,Eksplorasi Deskriptif tentang Layanan Bimbinga...,eksplorasi deskriptif tentang layanan bimbinga...,20,1
3,1,Pandemi Covid-19 merambah sejak awal tahun 202...,Perspektif Orang Tua terhadap Pelaksanaan Les ...,2021-06-02,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,Perspektif Orang Tua terhadap Pelaksanaan Les ...,perspektif orang tua terhadap pelaksanaan les ...,18,1
4,1,Dalam upaya mencapai Sustainable Development G...,Strategi Pencegahan dan Penanganan Stunting Mu...,2023-12-26,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,Strategi Pencegahan dan Penanganan Stunting Mu...,strategi pencegahan dan penanganan stunting mu...,17,1
...,...,...,...,...,...,...,...,...,...
8673,12,The purpose of this study to examine the effec...,"PENGARUH KINERJA KEUANGAN, DANA ALOKASI UMUM D...",2017-05-15,E-JURNAL AKUNTANSI,"PENGARUH KINERJA KEUANGAN, DANA ALOKASI UMUM D...",pengaruh kinerja keuangan dana alokasi umum da...,14,12
8674,12,The level of understanding of accounting to be...,"PENGARUH KECERDASAN EMOSIONAL, KECERDASAN INTE...",2017-08-06,E-JURNAL AKUNTANSI,"PENGARUH KECERDASAN EMOSIONAL, KECERDASAN INTE...",pengaruh kecerdasan emosional kecerdasan intel...,14,12
8675,12,The purpose of this study was to examine the i...,"PENGARUH UKURAN PERUSAHAAN, KOMPLEKSITAS OPERA...",2017-02-15,E-JURNAL AKUNTANSI,"PENGARUH UKURAN PERUSAHAAN, KOMPLEKSITAS OPERA...",pengaruh ukuran perusahaan kompleksitas operas...,14,12
8676,12,The purpose of this study is to determine the ...,KINERJA DINAS PARIWISATA BALI BERDASARKAN KONS...,2016-11-10,E-JURNAL AKUNTANSI,KINERJA DINAS PARIWISATA BALI BERDASARKAN KONS...,kinerja dinas pariwisata bali berdasarkan kons...,11,12


In [3]:
df[df.desc == 'KOSONG']

,jid,desc,title,date,journal,data,data_cleaned,word,jid_umum


In [4]:
df.dropna(inplace=True)
# dataset.drop(columns=['Unnamed: 0'], inplace=True)
df = df[df.desc != 'KOSONG']
df = df[df.title != 'KOSONG']
df

,jid,desc,title,date,journal,data,data_cleaned,word,jid_umum
0,1,Belakangan ini Lembaga PAUD berbondong-bondong...,Kemampuan Bahasa Inggris Awal pada Periode Lin...,2023-10-05,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,Kemampuan Bahasa Inggris Awal pada Periode Lin...,kemampuan bahasa inggris awal pada periode lin...,17,1
1,1,The assessment process carried out in schools ...,Strengthening Early Childhood Learning Outcome...,2022-06-09,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,Strengthening Early Childhood Learning Outcome...,strengthening early childhood learning outcome...,17,1
2,1,Kewajiban guru PAUD untuk memberikan layanan b...,Eksplorasi Deskriptif tentang Layanan Bimbinga...,2023-06-07,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,Eksplorasi Deskriptif tentang Layanan Bimbinga...,eksplorasi deskriptif tentang layanan bimbinga...,20,1
3,1,Pandemi Covid-19 merambah sejak awal tahun 202...,Perspektif Orang Tua terhadap Pelaksanaan Les ...,2021-06-02,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,Perspektif Orang Tua terhadap Pelaksanaan Les ...,perspektif orang tua terhadap pelaksanaan les ...,18,1
4,1,Dalam upaya mencapai Sustainable Development G...,Strategi Pencegahan dan Penanganan Stunting Mu...,2023-12-26,JURNAL OBSESI: JURNAL PENDIDIKAN ANAK USIA DINI,Strategi Pencegahan dan Penanganan Stunting Mu...,strategi pencegahan dan penanganan stunting mu...,17,1
...,...,...,...,...,...,...,...,...,...
8673,12,The purpose of this study to examine the effec...,"PENGARUH KINERJA KEUANGAN, DANA ALOKASI UMUM D...",2017-05-15,E-JURNAL AKUNTANSI,"PENGARUH KINERJA KEUANGAN, DANA ALOKASI UMUM D...",pengaruh kinerja keuangan dana alokasi umum da...,14,12
8674,12,The level of understanding of accounting to be...,"PENGARUH KECERDASAN EMOSIONAL, KECERDASAN INTE...",2017-08-06,E-JURNAL AKUNTANSI,"PENGARUH KECERDASAN EMOSIONAL, KECERDASAN INTE...",pengaruh kecerdasan emosional kecerdasan intel...,14,12
8675,12,The purpose of this study was to examine the i...,"PENGARUH UKURAN PERUSAHAAN, KOMPLEKSITAS OPERA...",2017-02-15,E-JURNAL AKUNTANSI,"PENGARUH UKURAN PERUSAHAAN, KOMPLEKSITAS OPERA...",pengaruh ukuran perusahaan kompleksitas operas...,14,12
8676,12,The purpose of this study is to determine the ...,KINERJA DINAS PARIWISATA BALI BERDASARKAN KONS...,2016-11-10,E-JURNAL AKUNTANSI,KINERJA DINAS PARIWISATA BALI BERDASARKAN KONS...,kinerja dinas pariwisata bali berdasarkan kons...,11,12


In [5]:
df.iloc[0, 0]

1

In [6]:
df[df.word==1]

,jid,desc,title,date,journal,data,data_cleaned,word,jid_umum


In [7]:
class BertDataset(Dataset):
    def __init__(self, tokenizer,max_length):
        super(BertDataset, self).__init__()
        self.train_csv=pd.read_csv('data/data_sinta_cleaned.csv')
        self.tokenizer=tokenizer
        self.target=self.train_csv['jid']
        self.max_length=max_length
        
    def __len__(self):
        return len(self.train_csv)

    def __getitem__(self, index):
        
        text1 = self.train_csv.iloc[index,6]
        
        inputs = self.tokenizer.encode_plus(
            text1 ,
            None,
            padding='max_length',
            add_special_tokens=True,
            return_attention_mask=True,
            max_length=self.max_length,
            )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'target': torch.tensor(self.train_csv.iloc[index, 0], dtype=torch.long)
            }

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# model = AutoModel.from_pretrained('bert-base-multilingual-cased')

dataset = BertDataset(tokenizer, max_length=128)

dataloader=DataLoader(dataset=dataset,batch_size=32, shuffle=True)

In [9]:
class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.bert_model = AutoModel.from_pretrained("bert-base-multilingual-cased")
        self.out = nn.Linear(768, 12)
        
    def forward(self,ids,mask,token_type_ids):
        _,o2= self.bert_model(ids,attention_mask=mask,token_type_ids=token_type_ids, return_dict=False)
        
        out= self.out(o2)
        
        return out
    
model=BERT()

loss_fn = nn.CrossEntropyLoss()

#Initialize Optimizer
optimizer= optim.Adam(model.parameters(),lr= 0.0001)

In [10]:
def finetune(epochs,dataloader,model,loss_fn,optimizer):
    model.train()
    for  epoch in range(epochs):
        print(epoch)
        
        loop=tqdm(enumerate(dataloader),leave=False,total=len(dataloader))
        for batch, dl in loop:
            ids=dl['ids']
            token_type_ids=dl['token_type_ids']
            mask= dl['mask']
            label=dl['target']
            # label = label.unsqueeze(1)
            
            optimizer.zero_grad()
            
            output=model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids)

            # output = torch.argmax(output, 1)
            label = label.type_as(output)

            loss=loss_fn(output.long(),label.long())
            loss.backward()
            
            optimizer.step()
            
            pred = np.where(output >= 0, 1, 0)


            num_correct = sum(1 for a, b in zip(pred, label) if a[0] == b[0])
            num_samples = pred.shape[0]
            accuracy = num_correct/num_samples
            
            print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
            
            # Show progress while training
            loop.set_description(f'Epoch={epoch}/{epochs}')
            loop.set_postfix(loss=loss.item(),acc=accuracy)

    return model

In [11]:
model=finetune(2, dataloader, model, loss_fn, optimizer)

0


RuntimeError: "log_softmax_lastdim_kernel_impl" not implemented for 'Long'